In [ ]:
#python DLTools/SetupPython.py > mysetup.py
import mysetup
from LArTPCDNN.LoadData import *

# Number of Threads when reading and mixing data
n_threads=1

# Number of Threads when reading cached mixed data (2nd data pass)
n_threads2=1

# Number of batches read by each thread
multiplier=1


BatchSize=16
FileSearch="/data/LArIAT/h5_files_2D_3D/2D_h5/*.h5"

# Downsampling Factor
DownSampleSize=8

# Size of window scanned accross data
ScanWindowSize=256

# Normalize Data?
Normalize=True

NSamples=10000

# Energy Cut (unnecessary for latest datasets)
EnergyCut=0.61

# Particle types to read
Particles= ['electron', 'antielectron',
            'pion',             
            'photon',
            'pionPlus', 'pionMinus',
            'proton', 'antiproton',
            'muon', 'antimuon',
            'kaonMinus', 'kaonPlus']

NClasses=len(Particles)

if ScanWindowSize>0:
#    shapes=[(BatchSize*multiplier, 2, 240, ScanWindowSize), (BatchSize*multiplier, NClasses)]
    shapes=[(BatchSize*multiplier, 240, ScanWindowSize),
            (BatchSize*multiplier, 240, ScanWindowSize),
            (BatchSize*multiplier, NClasses)]
    viewshape=(None, 240, ScanWindowSize)
else:
    shapes=[(BatchSize*multiplier, 240, 4096/DownSampleSize),
            (BatchSize*multiplier, 240, 4096/DownSampleSize),
            (BatchSize*multiplier, NClasses)]

    viewshape=(None, 240, 4096/DownSampleSize)


# Separate Files into Training and Test samples
TrainSampleList,TestSampleList=DivideFiles(FileSearch,[.9,.1],
                                           datasetnames=[u'features'],
                                           Particles=Particles)

def MakeGenerator(SampleList,NSamples,
                  cachefile="LArIAT-LoadDataTest-Cache.h5",**kwargs):

    return DLMultiClassFilterGenerator(TrainSampleList, FilterEnergy(EnergyCut), max=NSamples,
                                       preprocessfunction=ProcessWireData(DownSampleSize,ScanWindowSize,Normalize),
                                       postprocessfunction=MergeInputs(),
                                       batchsize=BatchSize,
                                       shapes=shapes,
                                       n_threads=n_threads,
                                       multiplier=multiplier,
                                       cachefile=cachefile,
                                       **kwargs)

# Use DLGenerators to read data
Train_gen = MakeGenerator(TrainSampleList, NSamples,
                            cachefile="/tmp/LArTPCDNN-LArIAT-TrainEvent-Cache.h5")

gen=Train_gen.DiskCacheGenerator(n_threads2)



In [ ]:
# Get the first batch
Data=gen.next()

In [ ]:
Data[0][1].shape